# Taxi Demand Prediction - New York City

## Business/Real World Problem
For a given location in New York City, our goal is to <b>predict the average speed of a pickup from that given location</b>. 

As Taxi drivers in New York travel faster than 12miles/hr, they are paid more since their charge by the distance, in addition to any surcharges. As they go slower than 12miles/hr, they are charged by time. As a result, they benefit by going slower since it gaurantees a fixed income given that there is already traffic. The estimates of pickup speed could be transferred to their smartphones in real time.

There are two kinds of taxi for which we have data available

Yellow Taxi: Exclusively through street-hails. Can access all parts of New York
Green Taxi: Street Hail Livery (SHL)- Limited to certain part of New York

We are considering only the yellow taxis for the time period of ... . We have used ... data to make prediction for ... data.</b>

Credits to GauravTheP for his brilliant work on predicting number of pickups per location in New York, which we built upon for this work -> https://github.com/gauravtheP/Taxi-Demand-Prediction-New-York-City/

In [1]:
# !pip install -q gpxpy
# !pip install -q sklearn

In [2]:
# !pip install -q sklearn

In [3]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import folium
import gpxpy.geo
from datetime import datetime
import time
import seaborn as sns
import os
import math
import xgboost as xgb
import matplotlib
matplotlib.use('nbagg')
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.cluster import MiniBatchKMeans, KMeans


# from numba import jit, cuda 

import warnings
warnings.simplefilter('ignore')

In [4]:
# !wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-01.csv


In [5]:
# !wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-01.csv

In [6]:
!ls

ML_Project_3.ipynb		  yellow_tripdata_2015-01.csv
README.md			  yellow_tripdata_2016-01.csv
Taxi_Demand_Prediction_NYC.ipynb


In [7]:
data_2015 = dd.read_csv("yellow_tripdata_2015-01.csv")
# data_2015 = pd.read_csv("yellow_tripdata_2015-01.csv")

In [8]:
data_2015.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [9]:
data_2015.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [10]:
print("Number of columns = "+str(len(data_2015.columns)))

Number of columns = 19


In [11]:
# data_2015.visualize()

In [12]:
data_2015["passenger_count"].sum()
# data_2015["passenger_count"].sum().visualize()

dd.Scalar<series-..., dtype=int64>

In [13]:
print("Total Number of Passengers in January 2015 = "+str(data_2015["passenger_count"].sum(axis = 0).compute()))
# print("Total Number of Passengers in January 2015 = "+str(data_2015["passenger_count"].sum(axis = 0))) #for pd instead of dd

Total Number of Passengers in January 2015 = 21437303


In [14]:
print("Number of nan values = {}".format(data_2015.isnull().sum().compute()))
# print("Number of nan values = {}".format(data_2015.isnull().sum())) #for pd instead of dd

Number of nan values = VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
pickup_longitude         0
pickup_latitude          0
RateCodeID               0
store_and_fwd_flag       0
dropoff_longitude        0
dropoff_latitude         0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    3
total_amount             0
dtype: int64


Since we will not be using "improvement_surcharge" in our task, so nan values will not affect us. So, we are leaving it as it is.

### Pickup 

In [15]:
# #detecting the pickups latitude and longitudes which are outside NYC.
# outside_NYC = data_2015[((data_2015.pickup_latitude <= 40.5774) | (data_2015.pickup_longitude <= -74.15) | (data_2015.pickup_latitude >= 40.9176) | (data_2015.pickup_longitude >= -73.7004))]
# #latitude at equator is 0. Above equator latitude increases and becomes 90 at north pole. Below equator latitude decreases and
# #is negative and becomes -90 at south pole.
# #Longitude is 0 at United Kingdom(UK). To the right of UK, longitude increases positively and to the left of UK longitude decrease
# #and is negative.

# m = folium.Map(location = [40.5774, -73.7004], tiles = "Stamen Toner")

# outside_pickups = outside_NYC.head(25000)

# for i,j in outside_pickups.iterrows():
#     if j["pickup_latitude"] != 0:
#         folium.Marker([j["pickup_latitude"], j["pickup_longitude"]]).add_to(m)
# m

<b>Observation:</b>As you can see in the above map, there are many erroneous data points, which are in some other state, and some of them are even in Atlantic Ocean. All these erroneous data points will also be removed as a part of data cleaning process.

### Dropoff

In [16]:
# #detecting the dropoff latitude and longitudes which are outside NYC.
# outside_NYC = data_2015[((data_2015.dropoff_latitude <= 40.5774) | (data_2015.dropoff_longitude <= -74.15) | (data_2015.dropoff_latitude >= 40.9176) | (data_2015.dropoff_longitude >= -73.7004))]
# #latitude at equator is 0. Above equator latitude increases and becomes 90 at north pole. Below equator latitude decreases and
# #is negative and becomes -90 at south pole.
# #Longitude is 0 at United Kingdom(UK). To the right of UK, longitude increases positively and to the left of UK longitude decrease
# #and is negative.

# m = folium.Map(location = [40.5774, -73.7004], tiles = "Stamen Toner")

# outside_dropoff = outside_NYC.head(25000)

# for i,j in outside_dropoff.iterrows():
#     if j["dropoff_latitude"] != 0:
#         folium.Marker([j["dropoff_latitude"], j["dropoff_longitude"]]).add_to(m)
# m
# #documentation of folium: https://python-visualization.github.io/folium/docs-v0.6.0/quickstart.html#Getting-Started

<b>Observation:</b>As you can see in the above map, there are many erroneous data points, which are in some other state, and some of them are even in Atlantic Ocean. All these erroneous data points will also be removed as a part of data cleaning process.

### Creating new Dataframe with Trip duration and speed

In [17]:
def timeToUnix(t):
    #we have a time in the format "YYYY-MM-DD HH:MM:SS", which is a string
    change = datetime.strptime(t, "%Y-%m-%d %H:%M:%S") #this will convert the String time into datetime format
    t_tuple = change.timetuple() #this will convert the datetime formatted time into structured time
    return time.mktime(t_tuple) + 3600  #this will convert structured time into unix-time.
    #Now why, I have added 3600 in the above unix times. NOW, UNIX TIMESTAMP MEANS HOW MANY SECONDS HAVE ELAPSED SINCE 1 JAN 1970
    #(EPOCH) CALCULATED FROM THE REFERENCE OF GMT. I HAVE MADE THIS PROJECT IN GERMANY WHICH IS 1HR/3600SECS AHEAD OF GMT TIME, 
    #AND HERE "time.mktime()" FUNCTION RETURNS UNIX TIMESTAMP FROM THE REFERENCE OF LOCAL TIME. SO, THEREFORE, IN ORDER TO 
    #COMPENSATE FOR 1HR AHEAD, "time.mktime" SUBTRACTED 3600 SECONDS MEANS 1HR FROM UNIX TIME STAMP IN ORDER TO CATER TO 
    #LOCAL TIME. SO, THEREFORE, IF WE WANT OUR UNIX TIME TO BE EXACTLY EQUAL TO GMT TIME, WE HAVE TO ADD 3600 SECONDS 
    #MEANS 1HR TO UNIX TIME. lET SAY AT 12:00AM ON 1st JAN 1970, TIME ELAPSED AT GMT IS 0, THE TIME ELAPSED IN GERMANY IS 
    #3600SEC. NOW ON 1st JAN 2015, ELASPED SECONDS AT GMT IS 'X', SO THE EQUIVALENT ELAPSED SECONDS IN GERMANY WILL BE X+3600. 
    #NOW "time.mktime()" SUBTRACT THIS 3600 EXTRA IN GERMAN TIME WHICH WE HAVE TO ADD IN ORDER TO MAKE IT EQUAL TO GMT.

def dfWithTripTimes(df):
    startTime = datetime.now()
    duration = df[["tpep_pickup_datetime", "tpep_dropoff_datetime"]].compute()
    #duration = df[["tpep_pickup_datetime", "tpep_dropoff_datetime"]] #for pd instead of dd

    pickup_time = [timeToUnix(pkup) for pkup in duration["tpep_pickup_datetime"].values]
    dropoff_time = [timeToUnix(drpof) for drpof in duration["tpep_dropoff_datetime"].values]
#     trip_duration = []
#     for xy in zip(dropoff_time, pickup_time):
#         trip_duration.append(xy[0] - xy[1])
    trip_duration = (np.array(dropoff_time) - np.array(pickup_time))/float(60)  #trip duration in minutes
    
    NewFrame = df[['passenger_count','trip_distance','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','total_amount']].compute()
#     NewFrame = df[['passenger_count','trip_distance','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','total_amount']] #for pd instead of dd
    NewFrame["trip_duration"] = trip_duration
    NewFrame["pickup_time"] = pickup_time
    NewFrame["speed"] = (NewFrame["trip_distance"]/NewFrame["trip_duration"])*60  #speed in miles/hr
    
    print("Time taken for creation of dataframe is {}".format(datetime.now() - startTime))
    return NewFrame

In [18]:
def changingLabels(num):
    if num < 10**3:
        return num
    elif num>=10**3 and num < 10**6:
        return str(num/10**3)+"k"
    elif num>=10**6 and num < 10**9:
        return str(num/10**6) + "M"
    else:
        return str(num/10**9) + "B"

In [19]:
# 1420070400 : 2015-01-01 00:00:00   (Equivalent unix time)
# 1451606400 : 2016-01-01 00:00:00   (Equivalent unix time)

def pickup_10min_bins(dataframe, month, year):
    pickupTime = dataframe["pickup_time"].values
    unixTime = [1420070400, 1451606400]
    unix_year = unixTime[year-2015]
    time_10min_bin = [int((i - unix_year)/600) for i in pickupTime]
    dataframe["time_bin"] = np.array(time_10min_bin)
    return dataframe


## Data Preparation for Januray 2015

In [20]:
startTime = datetime.now()

print("PREPARATION OF JANUARY 2015 DATA.")
print("-"*35)

print("Number of columns = "+str(len(data_2015.columns)))
print("-"*35)

new_frame = dfWithTripTimes(data_2015)
print("New Frame for Jan 2015 creation done")
print("-"*35)

# Observations:Here, 0th percentile value of trip duration is negative(weird), also 99th percentile 
# value of trip duration is 46.75min, but 100th percentile value is 548555.633min(weird). 
# 0th and 100th percentile values are certainly an erroneous points.

# According to NYC Taxi and Limousine Commission regulations, the maximum allowed trip duration 
# in a 24hrs interval is 12 hrs.¶

new_frame_cleaned = new_frame[(new_frame.trip_duration>1) & (new_frame.trip_duration<720)]
print("Trip Duration Outliers removed")
print("-"*35)

# Observations: Here, 100th percentile value of a speed is 192 Million miles/hr which is (BIZZARE). 
# Furthermore, 99.9th percentile value of speed is 45.31miles/hr. So, we are removing all the data points 
# where speed is greater than 45.31miles/hr.
new_frame_cleaned = new_frame_cleaned[(new_frame_cleaned.speed>0) & (new_frame_cleaned.speed<45.31)]
print("Speed Outliers removed")
print("-"*35)


# Observation: Here, 99.9th percentile of trip distance is 22.58miles, however, 
# 100th percentile value is 258.9miles, which is very high. So, we are removing all 
# the data points where trip distance is greater than 23miles.
new_frame_cleaned = new_frame_cleaned[(new_frame_cleaned.trip_distance>0) & (new_frame_cleaned.trip_distance<23)]
print("Trip Distance Outliers removed")
print("-"*35)


# Observation:Here, 99.9th percentile fare amount of a trip is 86.6. 
# However, 100th percentile of a fare amount is 3Million which is bizzare.
# Therefore, we have removedall the data points where fare amount is more than 
# 99.9th percentile value.
new_frame_cleaned = new_frame_cleaned[(new_frame_cleaned.total_amount>0) & (new_frame_cleaned.total_amount<86.6)]
print("Total Amount Outliers removed")
print("-"*35)

# Removing Pickups outside NYC
new_frame_cleaned = new_frame_cleaned[(((new_frame_cleaned.pickup_latitude >= 40.5774) & (new_frame_cleaned.pickup_latitude <= 40.9176)) & ((new_frame_cleaned.pickup_longitude >= -74.15) & (new_frame_cleaned.pickup_longitude <= -73.7004)))]
print("Pickups outside of NYC are removed")
print("-"*35)

# Removing Dropoffs outside NYC
new_frame_cleaned = new_frame_cleaned[(((new_frame_cleaned.dropoff_latitude >= 40.5774) & (new_frame_cleaned.dropoff_latitude <= 40.9176)) & ((new_frame_cleaned.dropoff_longitude >= -74.15) & (new_frame_cleaned.dropoff_longitude <= -73.7004)))]
print("Dropoffs outside of NYC are removed")
print("-"*35)


# Fraction of points left after removing all the erroneous points and outlier points. 
# Points where pickups and dropoffs are outside of NYC are also removed.
print("Fraction of cleaned points",str(new_frame_cleaned.shape[0]/new_frame.shape[0]))

print("Total number of outliers and erroneous points removed = ",str(new_frame.shape[0] - new_frame_cleaned.shape[0]))

#Clustering -setup
coord = new_frame_cleaned[["pickup_latitude", "pickup_longitude"]].values
regions = MiniBatchKMeans(n_clusters = 30, batch_size = 10000).fit(coord)

#Clustering-assignment
new_frame_cleaned["pickup_cluster"] = regions.predict(new_frame_cleaned[["pickup_latitude", "pickup_longitude"]])
print("Pickup Clusters are assigned")
print("-"*35)

# Time Binning
jan_2015_data = pickup_10min_bins(new_frame_cleaned, 1, 2015)
print("Pickup time bins are assigned")
print("-"*35)

jan_2015_data.head()
print("There should be ((24*60)/10)*31 unique 10 minute time bins for the month of January 2015: ", str(len(np.unique(jan_2015_data["time_bin"]))))

#grouping by pickup cluster then time bin within pickup cluster
jan_2015_timeBin_groupBy = jan_2015_data[["pickup_cluster", "time_bin", "trip_distance"]].groupby(by = ["pickup_cluster", "time_bin"]).count()
print("Pickup cluster and time bins are grouped.")
print("-"*35)

jan_2015_timeBin_groupBy.head()

print("Done...")
print("-"*35)

print("Fraction of Total data left = "+str(new_frame_cleaned.shape[0]/new_frame.shape[0]))
print("Total Number of outliers removed = "+str(new_frame.shape[0] - new_frame_cleaned.shape[0]))
print("-"*35)

print("Total Time taken for execution of Jan 2016 data = "+str(datetime.now() - startTime))
print("-"*35)

PREPARATION OF JANUARY 2016 DATA.
-----------------------------------
Number of columns = 19
-----------------------------------
Time taken for creation of dataframe is 0:05:25.018274
New Frame for Jan 2015 creation done
-----------------------------------
Trip Duration Outliers removed
-----------------------------------
Speed Outliers removed
-----------------------------------
Trip Distance Outliers removed
-----------------------------------
Total Amount Outliers removed
-----------------------------------
Pickups outside of NYC are removed
-----------------------------------
Dropoffs outside of NYC are removed
-----------------------------------
Fraction of cleaned points 0.9680387130396095
Total number of outliers and erroneous points removed =  407474
Pickup Clusters are assigned
-----------------------------------
Pickup time bins are assigned
-----------------------------------
There should be ((24*60)/10)*31 unique 10 minute time bins for the month of January 2015:  4464
Pick

NameError: name 'startTime' is not defined

#### Plot speed after removing outliers and erroneous points

<b>Observation:</b> PDF plot shows that most trip speeds are just above 10mls/hr

In [ ]:
# plt.figure(figsize = (12,8))
# sns.kdeplot(new_frame_cleaned["trip_duration"].values, shade = True, cumulative = False)
# plt.tick_params(labelsize = 20)
# plt.xlabel("Trip Duration", fontsize = 20)
# plt.title("PDF of Trip Duration", fontsize = 20)
# plt.show()
plt.figure(figsize = (12,8))
sns.kdeplot(new_frame_cleaned["speed"].values, shade = True, cumulative = False)
plt.tick_params(labelsize = 20)
plt.xlabel("Trip Speed", fontsize = 20)
plt.title("PDF of Trip Speed", fontsize = 20)
plt.show()

#### Box plot of speed after removing outliers and erroneous points.

In [ ]:
# fig = plt.figure(figsize = (10,6))
# ax = sns.boxplot("speed", data = new_frame_cleaned, orient = "v")

# plt.tick_params(labelsize = 20)
# plt.ylabel("Speed(Miles/hr)", fontsize = 20)
# plt.show()

In [ ]:
Average_speed = sum(new_frame_cleaned.speed)/len(new_frame_cleaned.speed)
print("Average Speed of Taxis around NYC = "+str(Average_speed))
# The avg speed in Newyork speed is 12.45miles/hr, so a cab driver can travel 2 miles per 10min on avg.

In [ ]:
print("Speed of Taxis around NYC per 10 minutes = "+str(Average_speed/6)+" per 10 minutes.")

#### Trip Distance

<b>Observation:</b> Most of the pickups are concentrated in and around Manhattan district of New York

In [ ]:
# m = folium.Map(location = [40.9176, -73.7004], tiles = "Stamen Toner")

# pickups_within_NYC = new_frame_cleaned.sample(n = 500)

# for i,j in pickups_within_NYC.iterrows():
#     folium.Marker([j["pickup_latitude"], j["pickup_longitude"]]).add_to(m)
# m

<b>Observation:</b> Most of the dropoffs are concentrated in and around Manhattan district of New York

In [ ]:
# m = folium.Map(location = [40.9176, -73.7004], tiles = "Stamen Toner")

# dropoff_within_NYC = new_frame_cleaned.sample(n = 500)

# for i,j in dropoff_within_NYC.iterrows():
#     folium.Marker([j["dropoff_latitude"], j["dropoff_longitude"]]).add_to(m)
# m

#### Plotting cluster centers

In [ ]:
# centerOfRegions = regions.cluster_centers_
# noOfClusters = len(centerOfRegions)
# m = folium.Map(location = [40.9176, -73.7004], tiles = "Stamen Toner")

# for i in range(noOfClusters):
#     folium.Marker([centerOfRegions[i][0], centerOfRegions[i][1]], popup = (str(np.round(centerOfRegions[i][0], 2))+", "+str(np.round(centerOfRegions[i][1], 2)))).add_to(m)
# m

#### Plotting Regions in NYC

In [ ]:
NYC_latitude_range = (40.5774, 40.9176)
NYC_Longitude_range = (-74.15, -73.7004)
fig = plt.figure()
ax = fig.add_axes([0,0,1.5,1.5])
ax.scatter(x = new_frame_cleaned.pickup_longitude.values[:70000], y = new_frame_cleaned.pickup_latitude.values[:70000], c = new_frame_cleaned.pickup_cluster.values[:70000], cmap = "Paired", s = 5)
ax.set_xlim(-74.10, -73.72)
ax.set_ylim(40.5774, 40.9176)
ax.set_title("Regions in New York City")
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.show()
#Longitude values vary from left to right i.e., horizontally
#Latitude values vary from top to bottom means i.e., vertically

## Data Preparation for Januray 2016

In [ ]:
# Up till now we cleaned data and prepared data for the month of Jan 2015.

# now doing the same operations for the month of Jan 2016.

# 1. Get the dataframe which includes only required colums.
# 2. Add trip_duration, speed, unix time stamp of pickup_time.
# 4. Remove the outliers based on trip_duration, speed, trip_distance, total_amount.
# 5. Remove all the points where pickup and dropoff are outside of New York City area.
# 6. Add pickup_cluster to each data point.
# 7. Add time_bin (index of 10min intravel to which that trip belongs to).
# 8. Group by data, based on 'pickup_cluster' and 'time_bin'
startTime = datetime.now()
frame_2016 = dd.read_csv("yellow_tripdata_2016-01.csv")
# frame_2016 = pd.read_csv("yellow_tripdata_2016-01.csv")

print("PREPARATION OF JANUARY 2016 DATA.")
print("-"*35)

print("Number of columns = "+str(len(frame_2016.columns)))
print("-"*35)

new_frame2 = dfWithTripTimes(frame_2016)
print("New Frame for Jan 2016 creation done")
print("-"*35)

new_frame_cleaned2 = new_frame2[(new_frame2.trip_duration>1) & (new_frame2.trip_duration<720)]
print("Trip Duration Outliers removed")
print("-"*35)

new_frame_cleaned2 = new_frame_cleaned2[(new_frame_cleaned2.speed>0) & (new_frame_cleaned2.speed<45.31)]
print("Speed Outliers removed")
print("-"*35)

new_frame_cleaned2 = new_frame_cleaned2[(new_frame_cleaned2.trip_distance>0) & (new_frame_cleaned2.trip_distance<23)]
print("Trip Distance Outliers removed")
print("-"*35)

new_frame_cleaned2 = new_frame_cleaned2[(new_frame_cleaned2.total_amount>0) & (new_frame_cleaned2.total_amount<86.6)]
print("Total Amount Outliers removed")
print("-"*35)

new_frame_cleaned2 = new_frame_cleaned2[(((new_frame_cleaned2.pickup_latitude >= 40.5774) & (new_frame_cleaned2.pickup_latitude <= 40.9176)) & ((new_frame_cleaned2.pickup_longitude >= -74.15) & (new_frame_cleaned2.pickup_longitude <= -73.7004)))]
print("Pickups outside of NYC are removed")
print("-"*35)

new_frame_cleaned2 = new_frame_cleaned2[(((new_frame_cleaned2.dropoff_latitude >= 40.5774) & (new_frame_cleaned2.dropoff_latitude <= 40.9176)) & ((new_frame_cleaned2.dropoff_longitude >= -74.15) & (new_frame_cleaned2.dropoff_longitude <= -73.7004)))]
print("Dropoffs outside of NYC are removed")
print("-"*35)

new_frame_cleaned2["pickup_cluster"] = regions.predict(new_frame_cleaned2[["pickup_latitude", "pickup_longitude"]])
print("Pickup Clusters are assigned")
print("-"*35)

jan_2016_data = pickup_10min_bins(new_frame_cleaned2, 1, 2016)
print("Pickup time bins are assigned")
print("-"*35)

jan_2016_timeBin_groupBy = jan_2016_data[["pickup_cluster", "time_bin", "trip_distance"]].groupby(by = ["pickup_cluster", "time_bin"]).count()
print("Pickup cluster and time bins are grouped.")
print("-"*35)

print("Done...")
print("-"*35)

print("Fraction of Total data left = "+str(new_frame_cleaned2.shape[0]/new_frame2.shape[0]))
print("Total Number of outliers removed = "+str(new_frame2.shape[0] - new_frame_cleaned2.shape[0]))
print("-"*35)

print("Total Time taken for execution of Jan 2016 data = "+str(datetime.now() - startTime))
print("-"*35)

In [ ]:
jan_2016_data.head()

In [ ]:
jan_2016_timeBin_groupBy.head()

In [ ]:
jan_2015_data.head()

In [ ]:
jan_2015_timeBin_groupBy.head()
# jan_2015_timeBin_groupBy["trip_distance"].values

## Smoothing

In [ ]:
# Gets the unique time bins where pickup values are present for each region.

# for each cluster region we will collect all the indices of 10min intervals in which pickups happened.
# we got an observation that there are some time bins that doesn't have any pickups.

def getUniqueBinsWithPickups(dataframe):
    values = []
    for i in range(30):          #we have total 30 clusters
        cluster_id = dataframe[dataframe["pickup_cluster"] == i]
        unique_clus_id = list(set(cluster_id["time_bin"]))
        unique_clus_id.sort()   #inplace sorting
        values.append(unique_clus_id)
        print( dataframe["pickup_cluster"]])
#         print(dataframe["pickup_cluster"]==i)
#         print(dataframe[dataframe["pickup_cluster"]==i])


    return values


In [ ]:
#now for Jan-2015, we have to find out, how many time_bins are there where there is no pickup in any of the cluster region
unique_binswithPickup_Jan_2015 = getUniqueBinsWithPickups(jan_2015_data)
for i in range(30):             #we have total 30 clusters
    print("For cluster ID {}, total number of time bins with no pickup in this clutser region is {}".format(i, (4464 - len(unique_binswithPickup_Jan_2015[i]))))
    print("-"*90)

There are two ways to fill up these values:

* Fill the missing value with 0's.

In [ ]:
# Fill the missing value with 0's.
def fillMissingWithZero(numberOfPickups, correspondingTimeBin):
    ind = 0
    smoothed_regions = []
    for c in range(0, 30):
        smoothed_bins = []
        for t in range(4464):    #there are total 4464 time bins in both Jan-2015 & Feb-2016.
            if t in correspondingTimeBin[c]:   #if a time bin is present in "correspondingTimeBin" in cluster 'c', 
            #then it means there is a pickup, in this case, we are simply adding number of pickups, else we are adding 0.
                smoothed_bins.append(numberOfPickups[ind])
                ind += 1
            else:
                smoothed_bins.append(0)
        smoothed_regions.extend(smoothed_bins)
    return smoothed_regions
#above function performs the operation in this way: if in any cluster if there is no pickup in any of the 4464 time bins, then 
#it simply appends 0 in that missing time_bin else it adds the original number of pickups in that time_bins.

In [ ]:
def smoothing(numberOfPickups, correspondingTimeBin):
    ind = 0
    repeat = 0
    smoothed_region = []
    for cluster in range(0, 30):
        smoothed_bin = []
        for t1 in range(4464):
            if repeat != 0:   #this will ensure that we shall not fill the pickup values again which we already filled by smoothing
                repeat -= 1
            else:
                if t1 in correspondingTimeBin[cluster]:
                    smoothed_bin.append(numberOfPickups[ind])
                    ind += 1
                else:
                    if t1 == 0:           
    #<---------------------CASE-1:Pickups missing in the beginning------------------------>
                        for t2 in range(t1, 4464):
                            if t2 not in correspondingTimeBin[cluster]:
                                continue
                            else:
                                right_hand_limit = t2
                                smoothed_value = (numberOfPickups[ind]*1.0)/((right_hand_limit + 1)*1.0)
                                for i in range(right_hand_limit + 1):
                                    smoothed_bin.append(math.ceil(smoothed_value))
                                ind += 1
                                repeat = right_hand_limit - t1
                                
                    if t1 != 0:
                        right_hand_limit = 0
                        for t2 in range(t1, 4464):
                            if t2 not in correspondingTimeBin[cluster]:
                                continue
                            else:
                                right_hand_limit = t2
                                break
                        if right_hand_limit == 0:
    #<---------------------CASE-2: Pickups MISSING IN THE END------------------------------>
                            smoothed_value = (numberOfPickups[ind-1]*1.0)/(((4464 - t1)+1)*1.0)
                            del smoothed_bin[-1]
                            for i in range((4464 - t1)+1):
                                smoothed_bin.append(math.ceil(smoothed_value))
                            repeat = (4464 - t1) - 1    
    #<---------------------CASE-3: Pickups MISSING IN MIDDLE OF TWO VALUES----------------> 
                        else: 
                            smoothed_value = ((numberOfPickups[ind-1] + numberOfPickups[ind])*1.0)/(((right_hand_limit - t1)+2)*1.0)
                            del smoothed_bin[-1]
                            for i in range((right_hand_limit - t1)+2):
                                smoothed_bin.append(math.ceil(smoothed_value))
                            ind += 1
                            repeat = right_hand_limit - t1                        
        smoothed_region.extend(smoothed_bin)
    return smoothed_region

# when we multiply any integer with "1.0", then it will be converted into float.

In [ ]:
jan_2015_fillZero = fillMissingWithZero(jan_2015_timeBin_groupBy["trip_distance"].values, unique_binswithPickup_Jan_2015)
# here in jan_2015_timeBin_groupBy dataframe the "trip_distance" represents the number of pickups that are happened.
jan_2015_fillSmooth = smoothing(jan_2015_timeBin_groupBy["trip_distance"].values, unique_binswithPickup_Jan_2015)

#"unique_binswithPickup_Jan_2015" contains all the unique time bins, where pickup happened. It contains 30 sub-arrays as there are 30 clusters
#and each sub-array contains the unique ID of all the time bins where pickup happened in the clusters which is the index of that sub-array.

In [ ]:
def countZeros(num):
    count = 0
    for i in num:
        if i == 0:
            count += 1
    return count

In [ ]:
print("Number of values filled with zero in zero fill data= "+str(countZeros(jan_2015_fillZero)))

In [ ]:
print("Sanity check for number of zeros in smoothed data = "+str(countZeros(jan_2015_fillSmooth)))

There are total 30 clusters. Each cluster has 4464 time bins. After smoothing or fillWithZero, each of 4464 time bin has a pickup. So, there should be a total of <b>4464*30 = 133920</b> pickup values present for the month of January 2015. Let's Check, is it correct?

In [ ]:
print("Total number of pickup values = "+str(len(jan_2015_fillZero)))
print("Total number of pickup values = "+str(len(jan_2015_fillSmooth)))

#### Above plot is a plot of all the pickup values for cluster 26th. It shows that all the zero filled are separated by avg filled values. The minimum average filled value is 1.

In [ ]:
unique_binswithPickup_Jan_2016 = getUniqueBinsWithPickups(jan_2016_data)

In [ ]:
# Jan-2015 data is smoothed, Jan-2016 data missing values are filled with zero
jan_2016_fillZero = fillMissingWithZero(jan_2016_timeBin_groupBy["trip_distance"].values, unique_binswithPickup_Jan_2016)

In [ ]:
regionWisePickup_Jan_2016 = []
for i in range(30):
    regionWisePickup_Jan_2016.append(jan_2016_fillZero[4464*i:((4464*i)+4464)])
#"regionWisePickup_Jan_2016" is a list of lists which contains 30 sub lists, where the index of each sub-list is the 
#corresponding cluster number and the element of each sub-list is the pickup value. So, we know that there are 4464 time bins 
#in Jan 2016, hence, each sub-list is of size 4464.

In [ ]:
print(len(regionWisePickup_Jan_2016))
print(len(regionWisePickup_Jan_2016[0]))

In [ ]:
def find_missing_timeBins(IDs):
    missing = []
    for i in range(len(unique_binswithPickup_Jan_2015[IDs]) - 1):
        j = unique_binswithPickup_Jan_2015[IDs][i]
        k = unique_binswithPickup_Jan_2015[IDs][i+1]
        if (k-j) > 1:
            for l in range(j+1, k):
                missing.append(l)
    return missing

# Baseline Models

Now we get into modelling in order to forecast the pickup densities for the months of Jan-2016 for which we are using multiple models with two variations. 
1. Using Ratios of the 2016 data to the 2015 data i.e $R_{t} = P^{2016}_{t} / P^{2015}_{t}$
2. Using Previous known values of the 2016 data itself to predict the future values

### Preparing dataframe with $x_i$ as Jan- 2015 pickups and $y_i$ as Jan-2016 Pickups, with ratios as $Pickup^{2016} / Pickup^{2015}.$

In [ ]:
Ratios_DF = pd.DataFrame()
Ratios_DF["Given"] = jan_2015_fillSmooth
Ratios_DF["Prediction"] = jan_2016_fillZero
Ratios_DF["Ratio"] = Ratios_DF["Prediction"]*1.0/Ratios_DF["Given"]*1.0

In [ ]:
Ratios_DF.head()

In [ ]:
Ratios_DF.shape

In [ ]:
print("Total Number of zeros in Ratio column = "+str(Ratios_DF["Ratio"].value_counts()[0]))

In [ ]:
print("Total Number of zeros in Prediction column = "+str(Ratios_DF["Prediction"].value_counts()[0]))

## Simple Weighted Moving Average
The Moving Avergaes Model used gave equal importance to all the values in the window used, but we know intuitively that the future is more likely to be similar to the latest values and less similar to the older values. Weighted Averages converts this analogy into a mathematical relationship giving the highest weight while computing the averages to the latest previous value and decreasing weights to the subsequent older ones<br>

Weighted Moving Averages using Ratio Values: $R_t = (N*R_{t-1} + N-1*R_{t-2} + N-2*R_{T-3} + ...) / N*(N+1)/2$

For the above the Hyperparameter is the window-size (n) which is tuned manually and it is found that the window-size of 4 is optimal for getting the best results using Weighted Moving Averages using previous Ratio values therefore we get $ R_{t} = ( 4*R_{t-1} + 3*R_{t-2} + 2*R_{t-3} + 1*R_{t-4})/10$

Weighted Moving Averages using Previous 2016 Values - $P_{t} = ( N*P_{t-1} + (N-1)*P_{t-2} + (N-2)*P_{t-3} .... 1*P_{t-n} )/(N*(N+1)/2)$

In [ ]:
def weighted_moving_average_predictions(ratios):
    predicted_pickup = (ratios["Prediction"].values)[0]
    predicted_pickup_values = []
    absolute_error = []
    squared_error = []
    window_size = 2
    for i in range(4464*30):
        if i % 4464 == 0:
            predicted_pickup_values.append(0)
            absolute_error.append(0)
            squared_error.append(0)
        else:
            predicted_pickup_values.append(predicted_pickup)
            absolute_error.append(abs(predicted_pickup_values[i] - ratios["Prediction"].values[i]))
            
            error = math.pow(int(predicted_pickup_values[i] - ratios["Prediction"].values[i]), 2)
            squared_error.append(error)
            
        if (i+1)>=window_size:
            sumPickups = 0
            sumOfWeights = 0
            for j in range(window_size, 0, -1):
                sumPickups = sumPickups + j*(ratios["Prediction"].values)[i -window_size + j]
                sumOfWeights = sumOfWeights + j
            predicted_pickup = sumPickups/sumOfWeights
        else:
            sumPickups = 0
            sumOfWeights = 0
            for j in range(i+1, 0, -1):
                sumPickups += j*(ratios["Prediction"].values)[j-1]
                sumOfWeights += j
            predicted_pickup = sumPickups/sumOfWeights
    
    ratios["Weighted_Moving_Average_Predictions_Pred"] = predicted_pickup_values
    ratios["Weighted_Moving_Average_Predictions_AbsError"] = absolute_error
    mean_absolute_percentage_error = (sum(absolute_error)/len(absolute_error)) / (sum(ratios["Prediction"]) / len(ratios["Prediction"]))
    mean_sq_error = sum(squared_error)/len(squared_error)
    return ratios, mean_absolute_percentage_error, mean_sq_error

For the above the Hyperparameter is the window-size (n) which is tuned manually and it is found that the window-size of 2 is optimal for getting the best results using Weighted Moving Averages using previous Ratio values therefore we get $ P_t = ( 2*P_{t-1} + 1*P_{t-2})/2$

Here, above, alpha is a hyper-parameter which needs to be tuned manually. It is found that alpha = 0.5 gives lowest MAPE value.

In [ ]:
r4, mape4, mse4 = weighted_moving_average_predictions(Ratios_DF)

In [ ]:
error_table_baseline = pd.DataFrame(columns = ["Model", "MAPE(%)", "MSE"])

error_table_baseline = error_table_baseline.append(pd.DataFrame([["Weighted Moving Average Predictions", mape4*100, mse4]], columns = ["Model", "MAPE(%)", "MSE"]))

error_table_baseline.reset_index(drop = True, inplace = True)

In [ ]:
error_table_baseline.style.highlight_min(axis=0)

<b>Plese Note:-</b> The above comparisons are made using Jan 2015 and Jan 2016 only

<b>From the above error table it is inferred that the best forecasting model for our prediction would be:-</b>
$P_{t} = ( N*P_{t-1} + (N-1)*P_{t-2} + (N-2)*P_{t-3} .... 1*P_{t-n} )/(N*(N+1)/2)$ i.e Weighted Moving Averages Predictions using 2016 Values.

# Regression Models

### Preparing Data

Preparing data to be split into train and test, The below code prepares data in cumulative form which will be later split into
test and train

There are total 30 clusters and for the month of January-2016 and there are total 4464 time bins. 
For each cluster region there are 4464 time bins and so, for 30 clusters there will be 4464*30 pickup values because after 
smoothing each time bin has pickup.
We will have a total of 4464*30 = 133920 pickup values.

In [ ]:
# we take number of pickups that are happened in last 5 10min intravels
number_of_time_stamps = 5

# TruePickups varaible
# it is list of lists
# It will be used as true labels/ground truth. Now since we are taking previous 5 pickups as a training data for predicting
# next pickup(here next pickup will be a true/ground truth pickup), so "TruePickups" will not contain first five pickups of each 
# cluster. It will contain number of pickups 4459 for each cluster. 
TruePickups = []

# lat will contain 4464-5=4459 times latitude of cluster center for every cluster.
# Ex: [[cent_lat 4459 times],[cent_lat 4459 times], [cent_lat 4459 times].... 30 lists]
# it is list of lists
lat = []

# lon will contain 4464-5=4459 times longitude of cluster center for every cluster.
# Ex: [[cent_lat 4459 times],[cent_lat 4459 times], [cent_lat 4459 times].... 30 lists]
# it is list of lists
lon = []

# we will code each day 
# sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5,sat=6
day_of_week = []


# for every cluster we will be adding 4459 values, each value represent to which day of the week that pickup bin belongs to
# it is list of lists

# feat is a numbpy array, of shape (133770, 5). {4459*30 = 133770.}
# each row corresponds to an entry in our data
# for the first row we will have [f0,f1,f2,f3,f4] fi=number of pickups happened in i+1st 10min intravel(bin)
# the second row will have [f1,f2,f3,f4,f5]
# the third row will have [f2,f3,f4,f5,f6]
# and so on...
feat = []


centerOfRegions = regions.cluster_centers_
feat = [0]*number_of_time_stamps
for i in range(30):
    lat.append([centerOfRegions[i][0]]*4459) 
    lon.append([centerOfRegions[i][1]]*4459)
    #1 January 2016 is a Friday so we start our day from 5: "(int(j/144))%7+5"
    # Our prediction starts from 5th 10min interval since we need to have number of pickups that are happened in last 5 pickup bins.
    day_of_week.append([int(((int(j/144)%7)+5)%7) for j in range(5, 4464)])
    #"regionWisePickup_Jan_2016" is a list of lists which contains 30 sub lists, where the index of each sub-list is the 
    #corresponding cluster number and the element of each sub-list is the pickup value. So, we know that there are 4464 time bins 
    #in Jan 2016, hence, each sub-list is of size 4464.
    #"regionWisePickup_Jan_2016" is a list of lists [[x1,x2,x3..4464], [x1,x2,x3..4464], [x1,x2,x3..4464], [x1,x2,x3..4464], [x1,x2,x3..4464], .. 30 lists]
    #Here, x1,x2,x3... are pickup values at time stamp 1,2,3... respectively
    feat = np.vstack((feat, [regionWisePickup_Jan_2016[i][k:k+number_of_time_stamps] for k in range(0, len(regionWisePickup_Jan_2016[i]) - (number_of_time_stamps))]))
    TruePickups.append(regionWisePickup_Jan_2016[i][5:])
    #output contains pickup values of all the regions and of each time stamp, except first 5 time stamp pickups of each region.
feat = feat[1:]

In [ ]:
len(lat[0])*len(lat) == len(lon[0])*len(lon) == len(day_of_week[0])*len(day_of_week) == 4459*30 == len(feat) == len(TruePickups[0])*len(TruePickups)

In [ ]:
feat

### Adding Predictions of Weighted Moving Average Predictions as a feature in our data

Getting the predictions of weighted moving averages to be used as a feature in cumulative form.

Upto now we computed 8 features for every data point that starts from 50th min of the day.
1. cluster center latitude
2. cluster center longitude
3. day of the week 
4. f_t_1: number of pickups that are happened previous t-1st 10min interval
5. f_t_2: number of pickups that are happened previous t-2nd 10min interval
6. f_t_3: number of pickups that are happened previous t-3rd 10min interval
7. f_t_4: number of pickups that are happened previous t-4th 10min interval
8. f_t_5: number of pickups that are happened previous t-5th 10min interval

From the baseline models we said that the weighted moving avarage predictions gives us the best error.
We will try to add the same weighted moving avarage predictions at time t as a feature to our data.<br>
Weighted Moving Average -> $P_{t} = ( N*P_{t-1} + (N-1)*P_{t-2} + (N-2)*P_{t-3} .... 1*P_{t-n} )/(N*(N+1)/2)$

In [ ]:
new_frame_cleaned.head()# "predicted_pickup_values": it is a temporary array that store weighted moving avarag prediction values for each 10min intervl, 
# for each cluster it will get reset.
# for every cluster it contains 4464 values
predicted_pickup_values = []

# "predicted_pickup_values_list"
# it is list of lists
# predict_list is a list of lists [[x5,x6,x7..x4463], [x5,x6,x7..x4463], [x5,x6,x7..x4463], ... 40 lists]
predicted_pickup_values_list = []

predicted_value = -1  #it will contain cuurent predicted_value. Default is given -1 which will be replaced later

window_size = 2
for i in range(30):
    for j in range(4464):
        if j == 0:
            predicted_value = regionWisePickup_Jan_2016[i][j]
            predicted_pickup_values.append(0)
        else:
            if j>=window_size:
                sumPickups = 0
                sumOfWeights = 0
                for k in range(window_size, 0, -1):
                    sumPickups += k*(regionWisePickup_Jan_2016[i][j -window_size + (k - 1)])
                    sumOfWeights += k
                predicted_value = int(sumPickups/sumOfWeights)
                predicted_pickup_values.append(predicted_value)
            else:
                sumPickups = 0
                sumOfWeights = 0
                for k in range(j, 0, -1):
                    sumPickups += k*regionWisePickup_Jan_2016[i][k-1]
                    sumOfWeights += k
                predicted_value = int(sumPickups/sumOfWeights)
                predicted_pickup_values.append(predicted_value)
                
    predicted_pickup_values_list.append(predicted_pickup_values[5:])
    predicted_pickup_values = []

In [ ]:
len(predicted_pickup_values_list[0])*len(predicted_pickup_values_list) == 4459*30

### Adding Top 5 Frequencies and Amplitudes of Fourier Transform as features

In [ ]:
amplitude_lists = []
frequency_lists = []
for i in range(30):
    ampli  = np.abs(np.fft.fft(regionWisePickup_Jan_2016[i][0:4096]))
    freq = np.abs(np.fft.fftfreq(4096, 1))
    ampli_indices = np.argsort(-ampli)[1:]        #it will return an array of indices for which corresponding amplitude values are sorted in reverse order.
    amplitude_values = []
    frequency_values = []
    for j in range(0, 9, 2):   #taking top five amplitudes and frequencies
        amplitude_values.append(ampli[ampli_indices[j]])
        frequency_values.append(freq[ampli_indices[j]])
    for k in range(4459):    #those top 5 frequencies and amplitudes are same for all the points in one cluster
        amplitude_lists.append(amplitude_values)
        frequency_lists.append(frequency_values) 

<b>Now we have built our all the features. We have finally now following 19 features in our data:</b>
1. <b>f_t_1:</b> Number of pickups that are happened previous t-1st 10min interval
2. <b>f_t_2:</b> Number of pickups that are happened previous t-2nd 10min interval
3. <b>f_t_3:</b> Number of pickups that are happened previous t-3rd 10min interval
4. <b>f_t_4:</b> Number of pickups that are happened previous t-4th 10min interval
5. <b>f_t_5:</b> Number of pickups that are happened previous t-5th 10min interval 
6. <b>Freq1:</b> Fourier Frequency corresponding to 1st highest amplitude
7. <b>Freq2:</b> Fourier Frequency corresponding to 2nd highest amplitude
8. <b>Freq3:</b> Fourier Frequency corresponding to 3rd highest amplitude
9. <b>Freq4:</b> Fourier Frequency corresponding to 4th highest amplitude
10. <b>Freq5:</b> Fourier Frequency corresponding to 5th highest amplitude
11. <b>Amp1:</b>  Amplitude corresponding to 1st highest fourier transformed wave.
12. <b>Amp2:</b>  Amplitude corresponding to 2nd highest fourier transformed wave.
13. <b>Amp3:</b>  Amplitude corresponding to 3rd highest fourier transformed wave.
14. <b>Amp4:</b>  Amplitude corresponding to 4th highest fourier transformed wave.
15. <b>Amp5:</b>  Amplitude corresponding to 5th highest fourier transformed wave.
16. <b>Latitude:</b> Latitude of Cluster center.
17. <b>Longitude:</b> Longitude of Cluster Center.
18. <b>WeekDay:</b> Day of week of pickup.
19. <b>WeightedAvg:</b>: Weighted Moving Average Prediction values.

## Data Preparation for regression models
Before we start predictions using the tree based regression models we take Jan 2016 pickup data and split it such that for every region we have 80% data in train and 20% in test, ordered date-wise for every region.

In [ ]:
print("size of total train data :" +str(int(133770*0.8)))
print("size of total test data :" +str(int(133770*0.2)))

In [ ]:
print("size of train data for one cluster:" +str(int(4459*0.8)))
print("size of total test data for one cluster:" +str(int(4459*0.2)))

In [ ]:
train_previousFive_pickups  = [feat[i*4459:(4459*i+3567)] for i in range(30)]
test_previousFive_pickups  = [feat[(i*4459)+3567:(4459*(i+1))] for i in range(30)]

In [ ]:
train_fourier_frequencies = [frequency_lists[i*4459:(4459*i+3567)] for i in range(30)]
test_fourier_frequencies = [frequency_lists[(i*4459)+3567:(4459*(i+1))] for i in range(30)]

In [ ]:
train_fourier_amplitudes = [amplitude_lists[i*4459:(4459*i+3567)] for i in range(30)]
test_fourier_amplitudes = [amplitude_lists[(i*4459)+3567:(4459*(i+1))] for i in range(30)]

In [ ]:
print("Train Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of training points = {}".format(len(train_previousFive_pickups), len(train_previousFive_pickups[0]), len(train_previousFive_pickups)*len(train_previousFive_pickups[0])))
print("Test Data: Total number of clusters = {}. Number of points in each cluster = {}. Total number of test points = {}".format(len(test_previousFive_pickups), len(test_previousFive_pickups[0]), len(test_previousFive_pickups)*len(test_previousFive_pickups[0])))

In [ ]:
#taking 80% data as train data from each cluster
train_lat = [i[:3567] for i in lat]
train_lon = [i[:3567] for i in lon]
train_weekDay = [i[:3567] for i in day_of_week]
train_weighted_avg = [i[:3567] for i in predicted_pickup_values_list]
train_TruePickups = [i[:3567] for i in TruePickups]

In [ ]:
#taking 20% data as test data from each cluster
test_lat = [i[3567:] for i in lat]
test_lon = [i[3567:] for i in lon]
test_weekDay = [i[3567:] for i in day_of_week]
test_weighted_avg = [i[3567:] for i in predicted_pickup_values_list]
test_TruePickups = [i[3567:] for i in TruePickups]

In [ ]:
# convert from lists of lists of list to lists of list
train_pickups = []
test_pickups = []
train_freq = []
test_freq = []
train_amp = []
test_amp = []
for i in range(30):
    train_pickups.extend(train_previousFive_pickups[i])
    test_pickups.extend(test_previousFive_pickups[i])
    train_freq.extend(train_fourier_frequencies[i])
    test_freq.extend(test_fourier_frequencies[i])
    train_amp.extend(train_fourier_amplitudes[i])
    test_amp.extend(test_fourier_amplitudes[i])

In [ ]:
#stacking pickups,frequencies and amplitudes horizontally.

train_prevPickups_freq_amp = np.hstack((train_pickups, train_freq, train_amp))
test_prevPickups_freq_amp = np.hstack((test_pickups, test_freq, test_amp))

In [ ]:
print("Number of data points in train data = {}. Number of columns till now = {}".format(len(train_prevPickups_freq_amp), len(train_prevPickups_freq_amp[0])))
print("Number of data points in test data = {}. Number of columns till now = {}".format(len(test_prevPickups_freq_amp), len(test_prevPickups_freq_amp[0])))

In [ ]:
# converting lists of lists into single list i.e flatten

train_flat_lat = sum(train_lat, [])
train_flat_lon = sum(train_lon, [])
train_flat_weekDay = sum(train_weekDay, [])
train_weighted_avg_flat = sum(train_weighted_avg, [])
train_TruePickups_flat = sum(train_TruePickups, [])

test_flat_lat = sum(test_lat, [])
test_flat_lon = sum(test_lon, [])
test_flat_weekDay = sum(test_weekDay, [])
test_weighted_avg_flat = sum(test_weighted_avg, [])
test_TruePickups_flat = sum(test_TruePickups, [])

In [ ]:
#train dataframe
columns = ['ft_5','ft_4','ft_3','ft_2','ft_1', 'freq1', 'freq2','freq3','freq4','freq5', 'Amp1', 'Amp2', 'Amp3', 'Amp4', 'Amp5']
Train_DF = pd.DataFrame(data = train_prevPickups_freq_amp, columns = columns)
Train_DF["Latitude"] = train_flat_lat
Train_DF["Longitude"] = train_flat_lon
Train_DF["WeekDay"] = train_flat_weekDay
Train_DF["WeightedAvg"] = train_weighted_avg_flat

In [ ]:
#test dataframe
Test_DF = pd.DataFrame(data = test_prevPickups_freq_amp, columns = columns)
Test_DF["Latitude"] = test_flat_lat
Test_DF["Longitude"] = test_flat_lon
Test_DF["WeekDay"] = test_flat_weekDay
Test_DF["WeightedAvg"] = test_weighted_avg_flat

In [ ]:
print("Shape of train data = "+str(Train_DF.shape))
print("Shape of test data = "+str(Test_DF.shape))

In [ ]:
Train_DF.head()

In [ ]:
Test_DF.head()

## Linear Regression

In [ ]:
def lin_regression(train_data, train_true, test_data, test_true):
    
    #standardizing the data
    train_std = StandardScaler().fit_transform(train_data)
    test_std = StandardScaler().fit_transform(test_data)
    
    #hyper-paramater tuning
    clf = SGDRegressor(loss = "squared_loss", penalty = "l2")
    values = [10**-14, 10**-12, 10**-10, 10**-8, 10**-6, 10**-4, 10**-2, 10**0, 10**2, 10**4, 10**6]
    hyper_parameter = {"alpha": values}
    best_parameter = GridSearchCV(clf, hyper_parameter, scoring = "neg_mean_absolute_error", cv = 3)
    best_parameter.fit(train_std, train_true)
    alpha = best_parameter.best_params_["alpha"]
    
    #applying linear regression with best hyper-parameter
    clf = SGDRegressor(loss = "squared_loss", penalty = "l2", alpha = alpha)
    clf.fit(train_std, train_true)
    train_pred = clf.predict(train_std)
    train_MAPE = mean_absolute_error(train_true, train_pred)/ (sum(train_true)/len(train_true))
    train_MSE = mean_squared_error(train_true, train_pred)
    test_pred = clf.predict(test_std)
    test_MAPE = mean_absolute_error(test_true, test_pred)/ (sum(test_true)/len(test_true))
    test_MSE = mean_squared_error(test_true, test_pred)
    
    return train_MAPE, train_MSE, test_MAPE, test_MSE

## XGBoost Regressor

In [ ]:
def xgboost_reg(train_data, train_true, test_data, test_true):
    #hyper-parameter tuning
    hyper_parameter = {"max_depth":[1, 2, 3, 4], "n_estimators":[40, 80, 150, 600]}
    clf = xgb.XGBRegressor()
    best_parameter = GridSearchCV(clf, hyper_parameter, scoring = "neg_mean_absolute_error", cv = 3)
    best_parameter.fit(train_data, train_true)
    estimators = best_parameter.best_params_["n_estimators"]
    depth = best_parameter.best_params_["max_depth"]
    
    #applying xgboost regressor with best hyper-parameter
    clf = xgb.XGBRegressor(max_depth = depth, n_estimators = estimators)
    clf.fit(train_data, train_true)
    train_pred = clf.predict(train_data)
    train_MAPE = mean_absolute_error(train_true, train_pred)/ (sum(train_true)/len(train_true))
    train_MSE = mean_squared_error(train_true, train_pred)
    test_pred = clf.predict(test_data)
    test_MAPE = mean_absolute_error(test_true, test_pred)/ (sum(test_true)/len(test_true))
    test_MSE = mean_squared_error(test_true, test_pred)
    
    return train_MAPE, train_MSE, test_MAPE, test_MSE

In [ ]:
trainMAPE_lr, trainMSE_lr, testMAPE_lr, testMSE_lr = lin_regression(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)

trainMAPE_xgb, trainMSE_xgb, testMAPE_xgb, testMSE_xgb = xgboost_reg(Train_DF, train_TruePickups_flat, Test_DF, test_TruePickups_flat)

# Model Comparison

In [ ]:
error_table_regressions = pd.DataFrame(columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TestMAPE(%)", "TestMSE"])

error_table_regressions = error_table_regressions.append(pd.DataFrame([["Linear Regression", trainMAPE_lr*100, trainMSE_lr, testMAPE_lr*100, testMSE_lr ]], columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TestMAPE(%)", "TestMSE"]))

error_table_regressions = error_table_regressions.append(pd.DataFrame([["XGBoost Regressor", trainMAPE_xgb*100, trainMSE_xgb, testMAPE_xgb*100, testMSE_xgb]], columns = ["Model", "TrainMAPE(%)", "TrainMSE", "TestMAPE(%)", "TestMSE"]))
error_table_regressions.reset_index(drop = True, inplace = True)

In [ ]:
error_table_regressions

In [ ]:
Final_Table = pd.DataFrame(columns = ["Model", "Mean_Absolute_Per_Error(%)"])

Final_Table = Final_Table.append(pd.DataFrame([["Weighted Moving Average Predictions", mape4*100]], columns = ["Model", "Mean_Absolute_Per_Error(%)"]))

Final_Table = Final_Table.append(pd.DataFrame([["Linear Regression", testMAPE_lr*100]], columns = ["Model", "Mean_Absolute_Per_Error(%)"]))

Final_Table = Final_Table.append(pd.DataFrame([["XGBoost Regressor", testMAPE_xgb*100]], columns = ["Model", "Mean_Absolute_Per_Error(%)"]))

Final_Table.reset_index(drop = True, inplace = True)

In [ ]:
ax = Final_Table.plot(x = "Model", kind = "bar", figsize = (12, 8), grid = True, fontsize = 15)
ax.set_title("Test MAPE of all Models", fontsize = 25)
ax.set_ylabel("Mean Absolute Percentage Error(%)", fontsize = 15)

for i in ax.patches:   #ax.patches is an array which gives x position, y position, width of a bar graphs.
    ax.text(i.get_x()-.05, i.get_height()+0.19, str(round(i.get_height(), 2))+'%', fontsize=14, color='black')
#     ax.text(x, y, annotate_text, font_size, color)
plt.show()

In [ ]:
Final_Table.style.highlight_min(axis=0)